In [7]:
import json
import numpy
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(numpy.arange(17503), test_size=0.1)

In [9]:
test.sort()

In [10]:
test

array([    3,     4,    12, ..., 17474, 17492, 17496])

In [11]:
train.sort()

In [12]:
dct = {}
lst = []
for x in test:
    dct[x] = 0
for x in train:
    dct[x] = 1
for x in sorted(dct):
    lst.append(dct[x])

In [41]:
len(lst)

17503

In [42]:
fopen = open('train_test_split.txt', 'w')
print(lst, file=fopen)

In [19]:
from scipy.io import loadmat, savemat
from PIL import Image
import os
import os.path as osp
import numpy as np
import json

In [45]:
def check_empty(list,name):
    
    try:
        list[name]
    except ValueError:
        return True

    if len(list[name]) > 0:
        return False
    else:
        return True


db_type = 'train' # train, test
annot_file = loadmat('mpii_human_pose_v1_u12_1')['RELEASE']
save_path = 'val.json'

joint_num = 16
img_num = len(annot_file['annolist'][0][0][0])

aid = 0
counter = -1
coco = {'images': [], 'categories': [], 'annotations': []}
for img_id in range(img_num):
   # print(img_id)
    
    if (annot_file['img_train'][0][0][0][img_id] == 0):
        continue
    
    
    
    if ((db_type == 'train' and annot_file['img_train'][0][0][0][img_id] == 1) ) and \
        check_empty(annot_file['annolist'][0][0][0][img_id],'annorect') == False: #any person is annotated
        counter+=1
        if (lst[counter] == 1): # if val(0)
            continue
    
        filename = 'images/' + str(annot_file['annolist'][0][0][0][img_id]['image'][0][0][0][0]) #filename
        img = Image.open(filename)
        w,h = img.size
        img_dict = {'id': img_id, 'file_name': filename, 'width': w, 'height': h}
        coco['images'].append(img_dict)

        if db_type == 'test':
            continue
        
        person_num = len(annot_file['annolist'][0][0][0][img_id]['annorect'][0]) #person_num
        joint_annotated = np.zeros((person_num,joint_num))
        for pid in range(person_num):
            
            if check_empty(annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid],'annopoints') == False: #kps is annotated
                
                bbox = np.zeros((4)) # xmin, ymin, w, h
                kps = np.zeros((joint_num,3)) # xcoord, ycoord, vis

                #kps
                annot_joint_num = len(annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0])
                for jid in range(annot_joint_num):
                    annot_jid = annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0][jid]['id'][0][0]
                    kps[annot_jid][0] = annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0][jid]['x'][0][0]
                    kps[annot_jid][1] = annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0][jid]['y'][0][0]
                    kps[annot_jid][2] = 2
                
                
                
                keypoint_list = kps.reshape(-1).tolist()
                xmax = 0
                ymax = 0
                xmin = 10000
                ymin = 10000
                for i in range(16):
                    if (keypoint_list[i*3] < 0 or keypoint_list[i*3+1] < 0 or keypoint_list[i*3] > w or keypoint_list[i*3+1] > h):
                        keypoint_list[i*3]=0
                        keypoint_list[i*3+1]=0
                        keypoint_list[i*3+2]=0
                    else:
                        if (keypoint_list[i*3] > xmax):
                            xmax = keypoint_list[i*3] 
                        if (keypoint_list[i*3] < xmin):
                            xmin = keypoint_list[i*3] 
                        if (keypoint_list[i*3+1] > ymax):
                            ymax = keypoint_list[i*3+1] 
                        if (keypoint_list[i*3+1] < ymin):
                            ymin = keypoint_list[i*3+1] 
                
                width = xmax - xmin
                height = ymax - ymin
                # corrupted bounding box
                if width <= 0 or height <= 0:
                    continue
                # 20% extend    
                else:
                    bbox[0] = max(0, xmin)
                    bbox[1] = max(0, ymin)
                    bbox[2] = width
                    bbox[3] = height
  
                        
               # keypoint_list = changeOrderkpl(keypoint_list)
                person_dict = {'num_keypoints': int(np.sum(kps[:,2]==1)),'area': bbox[2]*bbox[3],'iscrowd': 0, 'keypoints': keypoint_list, 'image_id': img_id,'bbox': bbox.tolist(), 'category_id': 1, 'id': aid  }
                coco['annotations'].append(person_dict)
                aid += 1

category = {
    "supercategory": "person",
    "id": 1,  # to be same as COCO, not using 0
    "name": "person",
    "skeleton": [[0,1],
        [1,2], 
        [2,6], 
        [7,12], 
        [12,11], 
        [11,10], 
        [5,4], 
        [4,3], 
        [3,6], 
        [7,13], 
        [13,14], 
        [14,15], 
        [6,7], 
        [7,8], 
        [8,9]] ,
    "keypoints": ["r_ankle", "r_knee","r_hip", 
                    "l_hip", "l_knee", "l_ankle",
                  "pelvis", "throax",
                  "upper_neck", "head_top",
                  "r_wrist", "r_elbow", "r_shoulder",
                  "l_shoulder", "l_elbow", "l_wrist"]}

coco['categories'] = [category]

with open(save_path, 'w') as f:
    json.dump(coco, f)

In [46]:
json_file = open('val.json') 
data = json.load(json_file)
print(len(data['images']))

1751


In [ ]:
### для существующего разбиения

In [47]:
fopen = open('train_test_split.txt')

In [48]:
lines = fopen.readlines()

In [49]:
line = list(map(int, lines[0][1:-2].split(', ')))

In [35]:
def check_empty(list,name):
    
    try:
        list[name]
    except ValueError:
        return True

    if len(list[name]) > 0:
        return False
    else:
        return True


db_type = 'train' # train, test
annot_file = loadmat('mpii_human_pose_v1_u12_1')['RELEASE']
save_path = 'train.json'

joint_num = 16
img_num = len(annot_file['annolist'][0][0][0])

aid = 0
counter = -1
coco = {'images': [], 'categories': [], 'annotations': []}
for img_id in range(img_num):
   # print(img_id)
    
    if (annot_file['img_train'][0][0][0][img_id] == 0):
        continue
    
    
    
    if ((db_type == 'train' and annot_file['img_train'][0][0][0][img_id] == 1) ) and \
        check_empty(annot_file['annolist'][0][0][0][img_id],'annorect') == False: #any person is annotated
        counter+=1
        if (line[counter] == 0): # if val(0)
            continue
    
        filename = 'images/' + str(annot_file['annolist'][0][0][0][img_id]['image'][0][0][0][0]) #filename
        img = Image.open(filename)
        w,h = img.size
        img_dict = {'id': img_id, 'file_name': filename, 'width': w, 'height': h}
        coco['images'].append(img_dict)

        if db_type == 'test':
            continue
        
        person_num = len(annot_file['annolist'][0][0][0][img_id]['annorect'][0]) #person_num
        joint_annotated = np.zeros((person_num,joint_num))
        for pid in range(person_num):
            
            if check_empty(annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid],'annopoints') == False: #kps is annotated
                
                bbox = np.zeros((4)) # xmin, ymin, w, h
                kps = np.zeros((joint_num,3)) # xcoord, ycoord, vis

                #kps
                annot_joint_num = len(annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0])
                for jid in range(annot_joint_num):
                    annot_jid = annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0][jid]['id'][0][0]
                    kps[annot_jid][0] = annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0][jid]['x'][0][0]
                    kps[annot_jid][1] = annot_file['annolist'][0][0][0][img_id]['annorect'][0][pid]['annopoints']['point'][0][0][0][jid]['y'][0][0]
                    kps[annot_jid][2] = 2
                
                
                
               
                keypoint_list = kps.reshape(-1).tolist()
                xmax = 0
                ymax = 0
                xmin = 10000
                ymin = 10000
                for i in range(16):
                    if (keypoint_list[i*3] < 0 or keypoint_list[i*3+1] < 0 or keypoint_list[i*3] > w or keypoint_list[i*3+1] > h):
                        keypoint_list[i*3]=0
                        keypoint_list[i*3+1]=0
                        keypoint_list[i*3+2]=0
                    else:
                        if (keypoint_list[i*3] > xmax):
                            xmax = keypoint_list[i*3] 
                        if (keypoint_list[i*3] < xmin):
                            xmin = keypoint_list[i*3] 
                        if (keypoint_list[i*3+1] > ymax):
                            ymax = keypoint_list[i*3+1] 
                        if (keypoint_list[i*3+1] < ymin):
                            ymin = keypoint_list[i*3+1] 
                width = xmax - xmin
                height = ymax - ymin
                # corrupted bounding box
                if width <= 0 or height <= 0:
                    continue
                # 20% extend    
                else:
                    bbox[0] = max(0, xmin)
                    bbox[1] = max(0, ymin)
                    bbox[2] = width
                    bbox[3] = height
  
                        
               # keypoint_list = changeOrderkpl(keypoint_list)
                person_dict = {'num_keypoints': int(np.sum(kps[:,2]==1)),'area': bbox[2]*bbox[3],'iscrowd': 0, 'keypoints': keypoint_list, 'image_id': img_id,'bbox': bbox.tolist(), 'category_id': 1, 'id': aid  }
                coco['annotations'].append(person_dict)
                aid += 1

category = {
    "supercategory": "person",
    "id": 1,  # to be same as COCO, not using 0
    "name": "person",
    "skeleton": [[0,1],
        [1,2], 
        [2,6], 
        [7,12], 
        [12,11], 
        [11,10], 
        [5,4], 
        [4,3], 
        [3,6], 
        [7,13], 
        [13,14], 
        [14,15], 
        [6,7], 
        [7,8], 
        [8,9]] ,
    "keypoints": ["r_ankle", "r_knee","r_hip", 
                    "l_hip", "l_knee", "l_ankle",
                  "pelvis", "throax",
                  "upper_neck", "head_top",
                  "r_wrist", "r_elbow", "r_shoulder",
                  "l_shoulder", "l_elbow", "l_wrist"]}

coco['categories'] = [category]

with open(save_path, 'w') as f:
    json.dump(coco, f)

In [ ]:
до чистки:

{0: 0,
 1: 1,
 2: 1,
 3: 1,
 4: 0,
 5: 8,
 6: 29,
 7: 47,
 8: 61,
 9: 520,
 10: 198,
 11: 142,
 12: 3578,
 13: 384,
 14: 3106,
 15: 888,
 16: 19919}

после:

{0: 0,
 1: 1,
 2: 1,
 3: 1,
 4: 0,
 5: 8,
 6: 31,
 7: 50,
 8: 84,
 9: 499,
 10: 198,
 11: 231,
 12: 3503,
 13: 457,
 14: 3054,
 15: 1053,
 16: 19712}